In [1]:
import pandas as pd
import geopandas as gpd
import os
from dash import Dash, html, dcc, Input, Output

from handlers.general_visualization import GeneralVisualization
from handlers.district_visualization import DistrictVisualization
from handlers.metro_visualization import MetroVisualization
from handlers.crime_visualization import CrimeVisualization
from handlers.general_use import add_borders

In [2]:
data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'pancho_clean')
listings = pd.read_csv(os.path.join(data_path, 'listings_merged.csv'))
calendar = pd.read_csv(os.path.join(data_path, 'calendar.csv'))
gdf = gpd.read_file(os.path.join(data_path, 'neighbourhoods.geojson'))
crimes = pd.read_csv(os.path.join(data_path, 'crimes.csv'))
metro = pd.read_csv(os.path.join(data_path, 'metro.csv'))

In [3]:
gv = GeneralVisualization(listings, gdf)
dv = DistrictVisualization(listings, gdf)
mv = MetroVisualization(listings, metro, gdf)
cv = CrimeVisualization(crimes, gdf)

In [4]:
distritos = list(listings['neighbourhood_group'].unique())

app = Dash(__name__, assets_folder='assets')

app.layout = html.Div(className='container', children=[
	html.H1('Datos de AirBnB en Madrid'),
	
	dcc.Tabs(children=[
		dcc.Tab(label='Información general', children=[
			html.Div(id='general-graphs')
		]),

		dcc.Tab(label='Información por distrito', children=[
			html.H4('Selecciona un distrito:'),
			dcc.Dropdown(
				id='distrito-dropdown-cloropleth',
				options=[{'label': distrito, 'value': distrito} for distrito in distritos],
				value='CENTRO'
			),
			html.Div(id='distrito-graph-cloropleth', className='map-container')
		]),

		dcc.Tab(label='Información con metro', children=[
            html.Div(id='metro-graphs')
		]),

		dcc.Tab(label='Crímenes', children=[
			html.Div(id='crimes-graphs')
		])
	])
])

@app.callback(
	Output('crimes-graphs', 'children'),
	Input('crimes-graphs', 'id')
)
def crimes_graphs(_):
	graphs = cv.get_all_graphs()
	map = cv.get_map()
	map_html = map._repr_html_()

	return html.Div([
		graphs,
		html.Iframe(
			srcDoc=map_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('metro-graphs', 'children'),
	Input('metro-graphs', 'id')
)
def metro_graphs(_):
	metro_map_borders = mv.get_metro_map(cloropleth=False)
	metro_map_borders_html = metro_map_borders._repr_html_()

	metro_map_cloro = mv.get_metro_map(cloropleth=True)
	metro_map_cloro_html = metro_map_cloro._repr_html_()

	return html.Div([
		html.Iframe(
			srcDoc=metro_map_borders_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=metro_map_cloro_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('distrito-graph-cloropleth', 'children'),
	Input('distrito-dropdown-cloropleth', 'value')
)
def district_graphs(distrito):
	district_info = dv.get_district_info(distrito)
	district_cloropleth = dv.get_district_cloropleth(distrito)
	heatmap = dv.get_district_heatmap(distrito)

	choropleth_html = district_cloropleth._repr_html_()
	heatmap_html = heatmap._repr_html_()
	
	return html.Div([
		district_info,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('general-graphs', 'children'),
	Input('general-graphs', 'id')
)
def general_graphs(_):
	graphs = gv.get_all_graphs()
	cloropleth_map = gv.get_madrid_cloropleth()
	heatmap = gv.get_madrid_heatmap()

	choropleth_html = cloropleth_map._repr_html_()
	heatmap_html = heatmap._repr_html_()
	
	return html.Div([
		graphs,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

if __name__ == '__main__':
	app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~/Desktop/PRVD-2/env/lib/python3.13/site-packages/pandas/core/indexes/base.py:3805, in Index.get_loc(
    self=Index(['neighbourhood', 'neighbourhood_group', 'geometry'], dtype='object'),
    key='LATITUDE'
)
   3804 try:
-> 3805     return self._engine.get_loc(casted_key)
        casted_key = 'LATITUDE'
        self = Index(['neighbourhood', 'neighbourhood_group', 'geometry'], dtype='object')
   3806 except KeyError as err:

File index.pyx:167, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:196, in pandas._libs.index.IndexEngine.get_loc()

File pandas/_libs/hashtable_class_helper.pxi:7081, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas/_libs/hashtable_class_helper.pxi:7089, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'LATITUDE'

The above exception was the direct cause of the